In [ ]:
import clips
import sys
sys.path.append('../../../src/')
from clips_util import print_facts, print_rules, print_templates, build_read_assert

In [ ]:
# create the CLIPS environment
env = clips.Environment()

# deftemplates and facts

deftemplates are used to define the attributes of objects in the CLIPS knowledge base. They are similar to C structures. The deftemplate has a label similar to a Class label and slot definitions similar to Class attributes in Python.

facts are used to represent known information in the knowledge base.

In [ ]:
# Create a deftemplate that defines the attributes (via slot assignments) of a person
# Note the use of SYMBOL types which are different than STRING types. Symbols are allow for 
# a predefined set of values (e.g. eye_color and hair_color)
DEFTEMPLATE_PERSON = """
(deftemplate person
    (slot name (type STRING))
    (slot age (type INTEGER))
    (slot eye_color (type SYMBOL) (allowed-symbols brown blue green))
    (slot hair_color (type SYMBOL) (allowed-symbols brown blonde black red white grey purple))
)
"""

# Build the template in the CLIPS environment
env.build(DEFTEMPLATE_PERSON)

In [ ]:
# currently there are no facts in the knowledge base
# but there are templates
print_facts(env)
print_templates(env)

In [ ]:
# Assert a fact that will be added to the CLIPS Knowledge Base
person_template = env.find_template('person')

# What happens if you enter an invalid value for eye_color or hair_color?
fact_John = person_template.assert_fact(name='John', age=28, 
                                        eye_color=clips.Symbol('blue'), 
                                        hair_color=clips.Symbol('brown'))

fact_Zoe = person_template.assert_fact(name='Zoe', age=88, 
                                        eye_color=clips.Symbol('blue'), 
                                        hair_color=clips.Symbol('grey'))
# Display the current facts
print_facts(env)

In [ ]:
# we can retract facts
fact_Zoe.retract()
print_facts(env)

In [ ]:
# resetting the environment removes asserted facts but NOT rules and templates
env.reset()
print_facts(env)
print_templates(env)

In [ ]:
# clearing the environment removes everything 
env.clear()
print_templates(env)

# rules

In [ ]:
# make sure the environment is clear
env.clear()

# define a template for emergency facts
DEFTEMPLATE_EMERGENCY = """
(deftemplate emergency
		(slot kind (type STRING))
		(slot patient_status (type STRING)))
"""
env.build(DEFTEMPLATE_EMERGENCY)

# define a rule that prints a statement when there is a an emergency fact with type arrhythmia and patient-status conscious
DEFRULE_CARDIAC_EMERGENCY = """
(defrule cardiac-emergency
		(emergency (kind "arrhythmia") (patient_status "conscious"))
=>
(println "Activate the Rapid Response Team "))
"""
env.build(DEFRULE_CARDIAC_EMERGENCY)

In [ ]:
# assert an emergency fact that matches the attributes of a cardiac emergency
emergency_template = env.find_template('emergency')
fact_cardiac_emergency = emergency_template.assert_fact(kind='arrhythmia', patient_status='conscious')
print_facts(env)
print_rules(env)

In [ ]:
# run the environment to fire rules if there are any matching facts that trigger rules
env.run()

In [ ]:
# note that even though the fact is still in the KB, the rule will not fire again w/out resetting the environment
print_facts(env)
env.run()

In [ ]:
# build a rule that prompts the user for input and asserts a fact

# IPython notebook redirects stdin which prevents using clips "read" function to get user input
# this function builds a read_assert function for the input clips environment.
# The build function, read_assert, uses the Python "input" function to read user input and converts string input
# to appropriate type. 
# The prompt_map is a dictionary where keys are of the form template_name:slot_name and values
# are the prompts to display to the user when requesting input
# Exceptions are handled in try / except block to address input type and allowed value errors
# **** see src/clips_util.py for the function definition 
prompt_map = {
    "emergency:kind": "Enter the type of emergency: ",
    "emergency:patient_status": "What is the patient status: "
     
}
build_read_assert(env, prompt_map)

DEFRULE_READ_EMERGENCY = """
(defrule read_emergency
; there are no conditions for this rule
; so it will always fire
    =>
    (read_assert emergency)
)
"""
env.build(DEFRULE_READ_EMERGENCY)

env.run()

print_facts(env)

# deffacts
It is often convenient to have facts that are retained in the clips KB when the environment is reset. 

In [ ]:
env.clear() # remove everything from the environment for this example

DEFTEMPLATE_PERSON = """
(deftemplate person
    			(slot name)
                (slot age))
"""

DEFFACTS_PEOPLE = """
(deffacts people "Create people on reset"
    ;person declaration
    (person (name "Jane") (age 80))
    (person (name "John") (age 10))
)
"""

env.build(DEFTEMPLATE_PERSON)
env.build(DEFFACTS_PEOPLE)

print_facts(env) # no facts until reset

# with reset, deffacts take hold
env.reset()
print_facts(env)
